In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = Net()

In [ ]:
import torch.optim as optim

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# number of epochs to train the model
n_epochs = 15

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    
    # train the model
    model.train()
    for data, target in train_loader:
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
        
    # print training statistics
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch+1, 
        train_loss
        ))

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimilarityCNN(nn.Module):
    def __init__(self):
        super(SimilarityCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        return x

# create instance of the CNN
cnn = SimilarityCNN()

# extract features from an image
features = cnn(image)

# calculate cosine similarity between two feature vectors
def cosine_similarity(vec1, vec2):
    return torch.dot(vec1, vec2) / (torch.norm(vec1) * torch.norm(vec2))

similarity = cosine_similarity(features1, features2)

In [ ]:
# classify image with classification model
output = model(image)
_, predicted = torch.max(output, 1)
confidence = output[0][predicted]

# if confidence is below threshold, use similarity detection model
if confidence < threshold:
    features = cnn(image)
    similarity = cosine_similarity(features, precious_features)
    if similarity > similarity_threshold:
        predicted = precious_class

In [ ]:
import torch
import torch.nn as nn

class LSH(nn.Module):
    def __init__(self, num_hashes, hash_size):
        super(LSH, self).__init__()
        self.num_hashes = num_hashes
        self.hash_size = hash_size
        self.hashes = nn.ParameterList([nn.Parameter(torch.randn(hash_size)) for _ in range(num_hashes)])

    def forward(self, x):
        # calculate hash values for each hash function
        hash_values = []
        for h in self.hashes:
            hash_values.append((x @ h).sign().long())
        return torch.cat(hash_values, dim=1)

# create LSH model with 10 hash functions and hash size 128
lsh = LSH(10, 128)

# generate random feature vectors to use as "precious" samples
precious_features = torch.randn(100, 128)

# calculate hash values for the precious samples
precious_hashes = lsh(precious_features)

# extract features from a new image
features = cnn(image)

# calculate hash values for the image
hashes = lsh(features)

# search for similar images by counting the number of hash functions that give the same value for both the image and a precious sample
similarity = (precious_hashes == hashes).sum(dim=1).float() / lsh.num_hashes

# find the most similar precious sample
_, most_similar = similarity.max(dim=0)

In [ ]:
# initialize lists to store softmax values and labels for incorrect predictions
incorrect_logits = []
incorrect_labels = []

# set model to evaluation mode
model.eval()

# turn off gradients for validation, saves memory and computations
with torch.no_grad():
    for data, target in val_loader:
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # get the predicted class from the maximum value in the output-probability tensor
        _, predicted = torch.max(output, 1)
        # save softmax values and labels for incorrect predictions
        for i in range(len(predicted)):
            if predicted[i] != target[i]:
                incorrect_logits.append(output[i])
                incorrect_labels.append(target[i])

# convert lists to tensors
incorrect_logits = torch.stack(incorrect_logits)
incorrect_labels = torch.LongTensor(incorrect_labels)

 incorrect_logits and incorrect_labels contain the softmax values and labels, respectively, for all the incorrect predictions made by the model on the validation set. You can use these tensors to analyze the behavior of the model when it makes incorrect predictions.

 For example, you could calculate the average softmax values for each class and compare them to see which classes the model is most confident about when it makes incorrect predictions.

In [ ]:
# calculate average softmax values for each class
avg_logits = incorrect_logits.mean(dim=0)

# print average softmax values
for i in range(len(avg_logits)):
    print('Class {}: {:.3f}'.format(i, avg_logits[i]))